In [2]:
import pandas as pd
import numpy as np


#### Procesamiento de taxis

Para simular una carga incremental en la nube, el ultimo mes de datos no se utilizaran.

In [36]:
green_2301 = pd.read_parquet('descargas/green_tripdata_2023-01.parquet',engine='pyarrow')
green_2302 = pd.read_parquet('descargas/green_tripdata_2023-02.parquet',engine='pyarrow')
green_2303 = pd.read_parquet('descargas/green_tripdata_2023-03.parquet',engine='pyarrow')
green_2304 = pd.read_parquet('descargas/green_tripdata_2023-04.parquet',engine='pyarrow')
green_2305 = pd.read_parquet('descargas/green_tripdata_2023-05.parquet',engine='pyarrow')
green_2306 = pd.read_parquet('descargas/green_tripdata_2023-06.parquet',engine='pyarrow')
green_2307 = pd.read_parquet('descargas/green_tripdata_2023-07.parquet',engine='pyarrow')
green_2308 = pd.read_parquet('descargas/green_tripdata_2023-08.parquet',engine='pyarrow')
green_2309 = pd.read_parquet('descargas/green_tripdata_2023-09.parquet',engine='pyarrow')
green_2310 = pd.read_parquet('descargas/green_tripdata_2023-10.parquet',engine='pyarrow')
green_2311 = pd.read_parquet('descargas/green_tripdata_2023-11.parquet',engine='pyarrow')
green_2312 = pd.read_parquet('descargas/green_tripdata_2023-12.parquet',engine='pyarrow')
green_2401 = pd.read_parquet('descargas/green_tripdata_2024-01.parquet',engine='pyarrow')
green_2402 = pd.read_parquet('descargas/green_tripdata_2024-02.parquet',engine='pyarrow')
green_2403 = pd.read_parquet('descargas/green_tripdata_2024-03.parquet',engine='pyarrow')
green_2404 = pd.read_parquet('descargas/green_tripdata_2024-04.parquet',engine='pyarrow')
green_2405 = pd.read_parquet('descargas/green_tripdata_2024-05.parquet',engine='pyarrow')
green_2406 = pd.read_parquet('descargas/green_tripdata_2024-06.parquet',engine='pyarrow')
#green_2407 = pd.read_parquet('descargas/green_tripdata_2024-07.parquet',engine='pyarrow')

In [37]:
df_taxis_green = [green_2301,green_2302,green_2303,green_2304,green_2305,
               green_2306,green_2307,green_2308,green_2309,green_2310,green_2311,
               green_2312,green_2401,green_2402,green_2403,green_2404,green_2405,green_2406]
print(type(df_taxis_green))

<class 'list'>


In [38]:
# Concateamos para crear uno solo
df_taxis_green = pd.concat(df_taxis_green, ignore_index=True)
df_taxis_green.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126867 entries, 0 to 1126866
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1126867 non-null  int64         
 1   lpep_pickup_datetime   1126867 non-null  datetime64[us]
 2   lpep_dropoff_datetime  1126867 non-null  datetime64[us]
 3   store_and_fwd_flag     1057035 non-null  object        
 4   RatecodeID             1057035 non-null  float64       
 5   PULocationID           1126867 non-null  int64         
 6   DOLocationID           1126867 non-null  int64         
 7   passenger_count        1057035 non-null  float64       
 8   trip_distance          1126867 non-null  float64       
 9   fare_amount            1126867 non-null  float64       
 10  extra                  1126867 non-null  float64       
 11  mta_tax                1126867 non-null  float64       
 12  tip_amount             11268

1. **VendorID**: Un código que representa al proveedor de tecnología (TPEP) que proporcionó el registro del viaje.
   - `1`: Creative Mobile Technologies, LLC
   - `2`: VeriFone Inc.
2. **tpep_pickup_datetime**: Fecha y hora en que se inició el viaje (se activó el taxímetro).
3. **tpep_dropoff_datetime**: Fecha y hora en que finalizó el viaje (se desactivó el taxímetro).
4. **Passenger_count**: Cantidad de pasajeros reportados para el viaje. Este dato es ingresado manualmente por el conductor.
5. **Trip_distance**: Distancia recorrida durante el viaje, reportada en millas.
6. **PULocationID**: ID de la zona de taxi TLC donde comenzó el viaje.
7. **DOLocationID**: ID de la zona de taxi TLC donde terminó el viaje.
8. **RateCodeID**: Código de la tarifa aplicada para el viaje.
   - `1`: Tarifa estándar
   - `2`: JFK
   - `3`: Newark
   - `4`: Nassau o Westchester
   - `5`: Tarifa negociada
   - `6`: Viaje en grupo
9. **Store_and_fwd_flag**: Indica si el registro del viaje fue almacenado temporalmente en la memoria del vehículo debido a problemas de conectividad.
   - `Y`: Sí, almacenado y enviado más tarde
   - `N`: No, enviado en tiempo real
10. **Payment_type**: Método de pago utilizado.
    - `1`: Tarjeta de crédito
    - `2`: Efectivo
    - `3`: Sin cargo
    - `4`: Disputa
    - `5`: Desconocido
    - `6`: Viaje anulado
11. **Fare_amount**: Monto de la tarifa calculada en base al tiempo y distancia por el taxímetro.
12. **Extra**: Cargos adicionales (como cargos por hora pico o nocturnos).
13. **MTA_tax**: Un impuesto de $0.50 para la Autoridad Metropolitana de Transporte (MTA).
14. **Improvement_surcharge**: Un recargo de $0.30 que se aplica desde 2015.
15. **Tip_amount**: Propinas recibidas, automáticamente registradas para pagos con tarjeta de crédito (las propinas en efectivo no se incluyen).
16. **Tolls_amount**: Total de peajes pagados durante el viaje.
17. **Total_amount**: El total del viaje, excluyendo las propinas en efectivo.
18. **Congestion_Surcharge**: Recargo por congestión recolectado para el programa de tarifas de congestión del Estado de Nueva York.
19. **Airport_fee**: Un recargo de $1.25 para recogidas en los aeropuertos LaGuardia y JFK.

In [39]:
df_taxis_green.isna().sum()

VendorID                       0
lpep_pickup_datetime           0
lpep_dropoff_datetime          0
store_and_fwd_flag         69832
RatecodeID                 69832
PULocationID                   0
DOLocationID                   0
passenger_count            69832
trip_distance                  0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
ehail_fee                1126867
improvement_surcharge          0
total_amount                   0
payment_type               69832
trip_type                  69926
congestion_surcharge       69832
dtype: int64

#### Para lograr los objetivos de la empresa, se decidieron utilizar solamente las columnas de:

- pickup_datetime
- dropoff_datetime
- PULocationID      
- DOLocationID      
- passenger_count
- trip_distance
- tip_amount
- amount


In [40]:
drop_cols = ['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge',
                    'ehail_fee', 'congestion_surcharge', 'trip_type', 'VendorID', 'RatecodeID', 'payment_type']

# Eliminamos las columnas de la lista
df_taxis_green.drop(columns=drop_cols, inplace=True)
# Renombramos las columnas
df_taxis_green.rename(columns={'lpep_pickup_datetime': 'pickup_datetime','lpep_dropoff_datetime': 'dropoff_datetime',
                            'total_amount': 'amount'}, inplace=True)
df_taxis_green.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126867 entries, 0 to 1126866
Data columns (total 8 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   pickup_datetime   1126867 non-null  datetime64[us]
 1   dropoff_datetime  1126867 non-null  datetime64[us]
 2   PULocationID      1126867 non-null  int64         
 3   DOLocationID      1126867 non-null  int64         
 4   passenger_count   1057035 non-null  float64       
 5   trip_distance     1126867 non-null  float64       
 6   tip_amount        1126867 non-null  float64       
 7   amount            1126867 non-null  float64       
dtypes: datetime64[us](2), float64(4), int64(2)
memory usage: 68.8 MB


Las columnas de tip_amount y amount, se sumaran para tener una variable donde exprese la cantidad total del dinero que gasto el cliente.

In [41]:
# Sumamos las columnas amount y tip_amount para crear la columna total_price
df_taxis_green['total_price'] = df_taxis_green['amount'] + df_taxis_green['tip_amount']
df_taxis_green.drop(columns=['amount', 'tip_amount'], inplace=True)

Debido a que las columnas con cantidad de pasajeros y distancia del viaje pueden llegar a ser 0, se procedio a eliminar las filas que posean 0 en estas columnas.

In [42]:
print(df_taxis_green['passenger_count'].value_counts(),df_taxis_green['trip_distance'].value_counts())


passenger_count
1.0    891297
2.0     94581
5.0     27804
6.0     15144
3.0     13620
0.0      9260
4.0      5190
7.0        57
9.0        43
8.0        39
Name: count, dtype: int64 trip_distance
0.00        57261
1.40        10015
1.30         9529
1.20         8453
1.50         8092
            ...  
62.55           1
58.74           1
44.51           1
107.97          1
33967.47        1
Name: count, Length: 4283, dtype: int64


In [43]:
df_taxis_green = df_taxis_green[df_taxis_green['passenger_count'] != 0.0]
df_taxis_green = df_taxis_green[df_taxis_green['trip_distance'] != 0.0]

In [44]:
# Verificamos nulos
print(df_taxis_green['passenger_count'].isnull().sum())
# Debido a que sabemos que se efectuo el pago, completamos los valores nulos de la columna passenger_count con el valor 1.
df_taxis_green['passenger_count'].fillna(1, inplace=True)
df_taxis_green['passenger_count'].isnull().sum()

67546


0

#### Creacion de la columna "service" que sera booleana siendo 1 = "green" y 0 = "yellow"

In [45]:
df_taxis_green['service'] = 1
df_taxis_green.tail(2)

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,total_price,service
1126864,2024-06-30 19:39:00,2024-06-30 20:00:00,247,151,1.0,3.34,35.93,1
1126866,2024-06-30 22:10:00,2024-06-30 22:40:00,61,112,1.0,5.14,39.70,1


#### **Yellow Taxis**

In [46]:
yellow_2301 = pd.read_parquet('descargas/yellow_tripdata_2023-01.parquet',engine='pyarrow')
yellow_2302 = pd.read_parquet('descargas/yellow_tripdata_2023-02.parquet',engine='pyarrow')
yellow_2303 = pd.read_parquet('descargas/yellow_tripdata_2023-03.parquet',engine='pyarrow')
yellow_2304 = pd.read_parquet('descargas/yellow_tripdata_2023-04.parquet',engine='pyarrow')
yellow_2305 = pd.read_parquet('descargas/yellow_tripdata_2023-05.parquet',engine='pyarrow')
yellow_2306 = pd.read_parquet('descargas/yellow_tripdata_2023-06.parquet',engine='pyarrow')
yellow_2307 = pd.read_parquet('descargas/yellow_tripdata_2023-07.parquet',engine='pyarrow')
yellow_2308 = pd.read_parquet('descargas/yellow_tripdata_2023-08.parquet',engine='pyarrow')
yellow_2309 = pd.read_parquet('descargas/yellow_tripdata_2023-09.parquet',engine='pyarrow')
yellow_2310 = pd.read_parquet('descargas/yellow_tripdata_2023-10.parquet',engine='pyarrow')
yellow_2311 = pd.read_parquet('descargas/yellow_tripdata_2023-11.parquet',engine='pyarrow')
yellow_2312 = pd.read_parquet('descargas/yellow_tripdata_2023-12.parquet',engine='pyarrow')
yellow_2401 = pd.read_parquet('descargas/yellow_tripdata_2024-01.parquet',engine='pyarrow')
yellow_2402 = pd.read_parquet('descargas/yellow_tripdata_2024-02.parquet',engine='pyarrow')
yellow_2403 = pd.read_parquet('descargas/yellow_tripdata_2024-03.parquet',engine='pyarrow')
yellow_2404 = pd.read_parquet('descargas/yellow_tripdata_2024-04.parquet',engine='pyarrow')
yellow_2405 = pd.read_parquet('descargas/yellow_tripdata_2024-05.parquet',engine='pyarrow')
yellow_2406 = pd.read_parquet('descargas/yellow_tripdata_2024-06.parquet',engine='pyarrow')
#green_2407 = pd.read_parquet('descargas/yellow_tripdata_2024-07.parquet',engine='pyarrow')

In [47]:
df_taxis_yellow = [yellow_2301,yellow_2302,yellow_2303,yellow_2304,yellow_2305,yellow_2306,
                   yellow_2307,yellow_2308,yellow_2309,yellow_2310,yellow_2311,yellow_2312,
                   yellow_2401,yellow_2402,yellow_2403,yellow_2404,yellow_2405,yellow_2406]
print(type(df_taxis_yellow))

df_taxis_yellow = pd.concat(df_taxis_yellow, ignore_index=True)
df_taxis_yellow.info()

print(df_taxis_yellow.isna().sum())

<class 'list'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58642319 entries, 0 to 58642318
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airpo

In [48]:
drop_cols = ['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge',
                    'congestion_surcharge', 'VendorID', 'RatecodeID', 'payment_type','airport_fee','Airport_fee']

# Eliminamos las columnas de la lista
df_taxis_yellow.drop(columns=drop_cols, inplace=True)

# Renombramos las columnas
df_taxis_yellow.rename(columns={'tpep_pickup_datetime': 'pickup_datetime','tpep_dropoff_datetime': 'dropoff_datetime',
                            'total_amount': 'amount'}, inplace=True)
df_taxis_yellow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58642319 entries, 0 to 58642318
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[us]
 1   dropoff_datetime  datetime64[us]
 2   passenger_count   float64       
 3   trip_distance     float64       
 4   PULocationID      int64         
 5   DOLocationID      int64         
 6   tip_amount        float64       
 7   amount            float64       
dtypes: datetime64[us](2), float64(4), int64(2)
memory usage: 3.5 GB


- pickup_datetime
- dropoff_datetime
- PULocationID      
- DOLocationID      
- passenger_count
- trip_distance
- tip_amount
- amount

In [49]:
# Sumamos las columnas amount y tip_amount para crear la columna total_price
df_taxis_yellow['total_price'] = df_taxis_yellow['amount'] + df_taxis_yellow['tip_amount']
df_taxis_yellow.drop(columns=['amount', 'tip_amount'], inplace=True)

In [50]:
df_taxis_yellow = df_taxis_yellow[df_taxis_yellow['passenger_count'] != 0.0]
df_taxis_yellow = df_taxis_yellow[df_taxis_yellow['trip_distance'] != 0.0]

In [51]:
# Verificamos nulos
print(df_taxis_yellow['passenger_count'].isnull().sum())
# Debido a que sabemos que se efectuo el pago, completamos los valores nulos de la columna passenger_count con el valor 1.
df_taxis_yellow['passenger_count'].fillna(1, inplace=True)
df_taxis_yellow['passenger_count'].isnull().sum()

2869727


0

In [52]:
df_taxis_yellow['service'] = 0
df_taxis_yellow.tail(2)

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_price,service
58642317,2024-06-30 23:33:36,2024-06-30 23:42:37,1.0,1.60,158,231,18.41,0
58642318,2024-06-30 23:04:50,2024-06-30 23:11:15,1.0,1.28,158,186,16.40,0


In [53]:
new_order = ['pickup_datetime', 'dropoff_datetime', 'passenger_count',
             'trip_distance', 'PULocationID', 'DOLocationID','total_price','service']
df_taxis_green = df_taxis_green.reindex(columns=new_order)

In [54]:
# Concatenamos los DataFrames de yellow y green taxis
taxis = pd.concat([df_taxis_yellow, df_taxis_green], ignore_index=True)
taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57785408 entries, 0 to 57785407
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[us]
 1   dropoff_datetime  datetime64[us]
 2   passenger_count   float64       
 3   trip_distance     float64       
 4   PULocationID      int64         
 5   DOLocationID      int64         
 6   total_price       float64       
 7   service           int64         
dtypes: datetime64[us](2), float64(3), int64(3)
memory usage: 3.4 GB


In [55]:
taxis.pickup_datetime.min()

Timestamp('2001-01-01 00:06:49')

In [4]:
taxis = taxis.sort_values(by='pickup_datetime')

In [57]:
taxis.head(5)

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_price,service
7022616,2001-01-01 00:06:49,2001-01-01 15:42:11,2.0,7.41,161,231,53.90,0
12041332,2001-01-01 00:07:36,2001-01-01 00:16:31,1.0,1.68,239,43,14.70,0
11943398,2001-01-01 00:08:31,2001-01-01 16:01:27,1.0,18.63,132,264,115.22,0
8831805,2001-01-01 00:08:42,2001-01-01 14:13:51,1.0,17.68,132,68,81.80,0
12047517,2001-01-01 00:28:40,2001-01-01 01:11:09,1.0,10.23,239,97,52.50,0


Debido a la existencia de algunos datos que su fecha no corresponde con los años extraidos, se procedera a aplicar una mascara para su filtrado.

In [5]:
print(taxis.shape)
taxis = taxis[(taxis['pickup_datetime'].dt.year == 2023) | (taxis['pickup_datetime'].dt.year == 2024)]
print(taxis.shape)

(47089415, 16)
(47089377, 16)


In [30]:
taxis.tail(5)

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_price,service
56321563,2024-06-30 23:59:56,2024-07-01 00:01:30,1.0,0.65,162,162,14.14,0
56323027,2024-06-30 23:59:57,2024-07-01 00:17:15,1.0,9.22,138,236,71.87,0
56322210,2024-07-01 00:00:27,2024-07-01 00:08:00,6.0,1.72,48,163,15.00,0
56322398,2024-07-01 00:02:34,2024-07-01 00:21:07,1.0,9.38,138,49,46.55,0
56322147,2024-07-01 00:05:23,2024-07-01 00:15:02,5.0,2.07,263,151,26.70,0


In [58]:
taxis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57785408 entries, 7022616 to 55930135
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[us]
 1   dropoff_datetime  datetime64[us]
 2   passenger_count   float64       
 3   trip_distance     float64       
 4   PULocationID      int64         
 5   DOLocationID      int64         
 6   total_price       float64       
 7   service           int64         
dtypes: datetime64[us](2), float64(3), int64(3)
memory usage: 3.9 GB


In [59]:
from funciones import get_borough

In [60]:
#taxis = pd.read_parquet('Data\process_taxis.parquet',engine='pyarrow')
taxis['pickup_borough'] = taxis['PULocationID'].apply(get_borough)
taxis['dropoff_borough'] = taxis['DOLocationID'].apply(get_borough)

In [61]:
taxis.pickup_borough.value_counts()

pickup_borough
Manhattan        51012424
Queens            5621875
Brooklyn           626376
Unknown            400125
Bronx              120461
Staten Island        2499
EWR                  1648
Name: count, dtype: int64

In [62]:
taxis.dropoff_borough.value_counts()

dropoff_borough
Manhattan        51104271
Queens            3174826
Brooklyn          2321620
Unknown            641915
Bronx              364348
EWR                163501
Staten Island       14927
Name: count, dtype: int64

Filtro para evaluar los viajes que se realizaron solamente en Manhattan. 

In [63]:
taxis = taxis[(taxis['pickup_borough']=='Manhattan')& (taxis['dropoff_borough']=='Manhattan')]
taxis.drop(columns=['pickup_borough','dropoff_borough'],inplace=True)
taxis['passenger_count'] = taxis['passenger_count'].astype(int)
taxis.service.value_counts()

service
0    47054952
1      583478
Name: count, dtype: int64

In [64]:
print(taxis.shape)
taxis.info()

(47638430, 8)
<class 'pandas.core.frame.DataFrame'>
Index: 47638430 entries, 7022616 to 56322147
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[us]
 1   dropoff_datetime  datetime64[us]
 2   passenger_count   int32         
 3   trip_distance     float64       
 4   PULocationID      int64         
 5   DOLocationID      int64         
 6   total_price       float64       
 7   service           int64         
dtypes: datetime64[us](2), float64(2), int32(1), int64(3)
memory usage: 3.0 GB


In [5]:
# convert the trip_distance column from miles to km
taxis['trip_distance'] = taxis['trip_distance']*1.6

#### Para el calculo del presupuesto, se procedio a establecer cuales serian los taxis que utilizan gasolina, electricidad o hibridos
- 0 = vehiculo a gasolina
- 1 = vehiculo electrico
- 2 = vehiculo hibrido

In [66]:
proporciones = [0.5, 0.3, 0.2]
# Calcular el número de filas para cada categoría
num_filas = len(taxis)
num_por_categoria = [int(prop * num_filas) for prop in proporciones]
# Crear una lista con los valores de la nueva columna
fuel_type = []
for i in range(len(proporciones)):
    fuel_type.extend([i] * num_por_categoria[i])
# Mezclar aleatoriamente la lista
np.random.shuffle(fuel_type)

# Agregar la nueva columna al DataFrame
num_filas = len(taxis)
print(num_filas)

47638430


In [67]:
# Asegurarse de que la longitud de fuel_type sea igual a num_filas
assert len(fuel_type) == num_filas, "La longitud de fuel_type no coincide con el número de filas del dataframe."
taxis['fuel_type'] = fuel_type

Columna con la duracion del viaje

In [ ]:
taxis['trip_duration'] = (taxis['dropoff_datetime'] - taxis['pickup_datetime']).dt.total_seconds()

In [5]:
taxis.fuel_type.value_counts()

fuel_type
0    23819215
1    14291529
2     9527686
Name: count, dtype: int64

#### Consumo de energia y gasto

In [6]:
taxis['energy_consumption'] = np.where(
    taxis['fuel_type'] == 0,  # Condición para fuel_type == 0
    taxis['trip_distance'] * 0.1,  # Cálculo cuando fuel_type es 0
    np.where(
        taxis['fuel_type'] == 1,  # Condición para fuel_type == 1
        taxis['trip_distance'] * 0.025,  # Cálculo cuando fuel_type es 1
        taxis['trip_distance'] * 0.035  # Cálculo para otros casos, como fuel_type == 2
    )
)

Segun nuestras fuentes, el gasto en mantenimiento para autos electricos es aproximadamente un cuarto que el gasto de los vehiculos convencionales.
Para el desgaste del vehiculo convencional, si consideramos segun la AAA, el costo del mantenimiento y reparo del vehiculo es de 0.15 dolares por milla, lo cual serian 0.25 dolares por km aproximadamente sumadonse 0.0625(representando el desgaste de sus partes), 0.075 dolares por km para autos electricos y 0.15 dolares por km aproximadamente para autos hibridos.

In [3]:
taxis.drop(columns=['maintenance','net_profit'],inplace=True)

In [4]:
taxis['maintenance'] = np.where(
    taxis['fuel_type'] == 0,  # Condición para fuel_type == 0
    taxis['trip_distance'] * 0.3125,  
    np.where(
        taxis['fuel_type'] == 1,  # Condición para fuel_type == 1
        taxis['trip_distance'] * 0.075,  
        taxis['trip_distance'] * 0.15
    )
)

En New York, el salario mensual de un taxista puede oscilar entre $30.000 y $35.000, dependiendo de diversos factores. Si consideramos un promedio de $30.000 dolares anuales y una jornada laboral de 8 horas diarias, 6 días a la semana, el ingreso por hora estaria oscilando entre $13 a $15 dolares. Sin embargo, es importante tener en cuenta que este es un valor promedio y que los ingresos reales pueden variar considerablemente

Convertimos la duracion del viaje de segundos a horas

In [19]:
taxis["trip_duration"] = taxis["trip_duration"]/3600

CO2 emitido

In [3]:
taxis['co2'] = np.where(
    taxis['fuel_type'] == 0,  # Condición para fuel_type == 0
    taxis['trip_distance'] * 2.6,  
    np.where(
        taxis['fuel_type'] == 1,  # Condición para fuel_type == 1
        taxis['trip_distance'] * 0,  
        taxis['trip_distance'] * 0.99
    )
)

In [21]:
taxis[["trip_duration","dropoff_datetime","pickup_datetime","total_price"]].head(3)

,trip_duration,dropoff_datetime,pickup_datetime,total_price
7022616,15.589444,2001-01-01 15:42:11,2001-01-01 00:06:49,53.9
12041332,0.148611,2001-01-01 00:16:31,2001-01-01 00:07:36,14.7
7245608,6.830278,2001-01-01 22:08:40,2001-01-01 15:18:51,43.0


In [5]:
taxis["labor"] = taxis["trip_duration"] * 13
taxis["fuel_cost"] = taxis["energy_consumption"]*0.92
taxis["net_profit"] = taxis["total_price"]-taxis["fuel_cost"]-taxis["maintenance"]-taxis["labor"]

In [6]:
taxis[["trip_distance","fuel_type","fuel_cost","maintenance","labor","net_profit"]].sample(20)

,trip_distance,fuel_type,fuel_cost,maintenance,labor,net_profit
20457255,1.616,1,0.037168,0.1212,1.931944,11.209688
179039,0.640,0,0.058880,0.2000,0.711389,8.129731
4900359,29.520,2,0.950544,4.4280,9.630833,104.760623
620810,2.448,1,0.056304,0.1836,1.390278,16.989818
15659350,7.808,0,0.718336,2.4400,5.748889,33.742775
5119430,3.680,2,0.118496,0.5520,2.116111,22.133393
16058211,5.312,1,0.122176,0.3984,3.105556,26.753868
41256171,2.176,0,0.200192,0.6800,1.801944,17.337864
14135253,4.800,2,0.154560,0.7200,6.063056,39.862384
44992575,0.640,0,0.058880,0.2000,0.855833,12.985287


In [14]:
taxis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47097253 entries, 12041332 to 56322147
Data columns (total 16 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_datetime     datetime64[us]
 1   dropoff_datetime    datetime64[us]
 2   passenger_count     int32         
 3   trip_distance       float64       
 4   PULocationID        int64         
 5   DOLocationID        int64         
 6   total_price         float64       
 7   service             int64         
 8   fuel_type           int64         
 9   energy_consumption  float64       
 10  fuel_cost           float64       
 11  trip_duration       float64       
 12  labor               float64       
 13  co2                 float64       
 14  maintenance         float64       
 15  net_profit          float64       
dtypes: datetime64[us](2), float64(9), int32(1), int64(4)
memory usage: 5.8 GB


In [ ]:
print(taxis[taxis["net_profit"]<0].shape)
taxis[taxis["net_profit"]<0].sample(10)

In [37]:
print(taxis.loc[7022616])

pickup_datetime       2001-01-01 00:06:49
dropoff_datetime      2001-01-01 15:42:11
passenger_count                         2
trip_distance                      11.856
PULocationID                          161
DOLocationID                          231
total_price                          53.9
service                                 0
fuel_type                               0
energy_consumption                 1.1856
maintenance                        1.7784
fuel_cost                        1.090752
net_profit                     -151.63193
trip_duration                   15.589444
labor                          202.662778
Name: 7022616, dtype: object


In [36]:
print(taxis.loc[46494424])

pickup_datetime       2002-12-31 22:10:04
dropoff_datetime      2002-12-31 22:14:22
passenger_count                         1
trip_distance                       12.08
PULocationID                          170
DOLocationID                          231
total_price                         53.34
service                                 0
fuel_type                               0
energy_consumption                  1.208
maintenance                         1.812
fuel_cost                         1.11136
net_profit                      49.484973
trip_duration                    0.071667
labor                            0.931667
Name: 46494424, dtype: object


Considerando que existen ciertas inconsistencias en los datos como por ejemplo viajes excesivamente largo con lo que respecta al tiempo y no tan largos en distancia recorrida, teniendo en cuenta que un taxi si realiza un detenimiento durante su trayecto sigue cobrando por el viaje, aunque este parado y es por esto que:
Se filtro la data por:
- Valores negativos en total_price.
- Valores negativos en net_profit:
Esto se realizo debido a viajes por ejemplo de duracion muy elevada y valores de total_price negativos o sin sentido.
Tambien existen casos en los que la duracion fue extremadamente corta para la cantidad de kilometros recorridos durante el viaje(Se encontraron casos como viajes de 12 kilometros en 4 minutos siendo estos fisicamente imposible y probablemente un error al ingresar los datos)


In [6]:
print(taxis.shape)
taxis = taxis[taxis["total_price"]>0]
taxis = taxis[taxis["net_profit"]>0]
print(taxis.shape)

(47089377, 16)
(47089377, 16)


In [3]:
taxis = pd.read_parquet("Data/process_taxis.parquet",engine='pyarrow')

In [5]:
taxis.shape

(47097253, 16)

In [6]:
taxis.head(5)

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_price,service,fuel_type,energy_consumption,maintenance,fuel_cost,net_profit,trip_duration,labor,co2
12041332,2001-01-01 00:07:36,2001-01-01 00:16:31,1,2.688,239,43,14.70,0,1,0.06720,0.18816,0.061824,12.518072,0.148611,1.931944,0.00000
46494424,2002-12-31 22:10:04,2002-12-31 22:14:22,1,12.080,170,231,53.34,0,0,1.20800,1.81200,1.111360,49.484973,0.071667,0.931667,31.40800
44377670,2002-12-31 22:17:10,2002-12-31 22:42:24,1,2.240,50,162,21.00,0,0,0.22400,0.33600,0.206080,14.990698,0.420556,5.467222,5.82400
37020533,2002-12-31 22:59:39,2002-12-31 23:05:41,1,1.008,170,170,10.50,0,2,0.03528,0.10080,0.032458,9.059520,0.100556,1.307222,0.99792
12138808,2002-12-31 23:18:49,2002-12-31 23:33:34,1,4.848,141,79,21.00,0,1,0.12120,0.33936,0.111504,17.353303,0.245833,3.195833,0.00000


In [7]:
taxis.to_parquet('Data\process_taxis.parquet',engine='pyarrow')

In [76]:
del taxis
del df_taxis_green
del df_taxis_yellow

Fuentes:

https://www.moneygeek.com/insurance/auto/seguro-auto-barato-nueva-york/

https://www.miproximopaso.org/profile/wages/53-3054.00?s=NY

https://www.muchoneumatico.com/blog/coches/piezas-coche-desgastan/